In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import kabuki
import seaborn as sns
%matplotlib inline
import hddm
print(hddm.__version__)

0.8.0


/opt/homebrew/Caskroom/miniforge/base/envs/pyHDDM/lib/python3.6/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


In [ ]:
def read_models(filepath, n_chains=4):
    models=[]
    for i in range(n_chains):    
        models.append(hddm.load(f'{filepath}_{i}model'))
    return kabuki.utils.concat_models(models)

In [2]:
model_loa = read_models('./atvz_loa_4c_4000s_1000b', n_chains=4)
model_hia = read_models('./atvz_hia_4c_4000s_1000b', n_chains=4)

In [3]:
a_hia_101, a_hia_102 = model_hia.nodes_db.node[['a(101)', 'a(102)']]
a_loa_101, a_loa_102 = model_loa.nodes_db.node[['a(101)', 'a(102)']]

t_hia_101, t_hia_102 = model_hia.nodes_db.node[['t(101)', 't(102)']]
t_loa_101, t_loa_102 = model_loa.nodes_db.node[['t(101)', 't(102)']]

v_hia_101, v_hia_102 = model_hia.nodes_db.node[['v(101)', 'v(102)']]
v_loa_101, v_loa_102 = model_loa.nodes_db.node[['v(101)', 'v(102)']]

z_hia_101, z_hia_102 = model_hia.nodes_db.node[['z(101)', 'z(102)']]
z_loa_101, z_loa_102 = model_loa.nodes_db.node[['z(101)', 'z(102)']]

In [4]:
a_hia_101_mean, a_hia_101_std = a_hia_101.trace().mean(), a_hia_101.trace().std()
a_hia_102_mean, a_hia_102_std = a_hia_102.trace().mean(), a_hia_102.trace().std()

a_loa_101_mean, a_loa_101_std = a_loa_101.trace().mean(), a_loa_101.trace().std()
a_loa_102_mean, a_loa_102_std = a_loa_102.trace().mean(), a_loa_102.trace().std()

t_hia_101_mean, t_hia_101_std = t_hia_101.trace().mean(), t_hia_101.trace().std()
t_hia_102_mean, t_hia_102_std = t_hia_102.trace().mean(), t_hia_102.trace().std()

t_loa_101_mean, t_loa_101_std = t_loa_101.trace().mean(), t_loa_101.trace().std()
t_loa_102_mean, t_loa_102_std = t_loa_102.trace().mean(), t_loa_102.trace().std()

v_hia_101_mean, v_hia_101_std = v_hia_101.trace().mean(), v_hia_101.trace().std()
v_hia_102_mean, v_hia_102_std = v_hia_102.trace().mean(), v_hia_102.trace().std()

v_loa_101_mean, v_loa_101_std = v_loa_101.trace().mean(), v_loa_101.trace().std()
v_loa_102_mean, v_loa_102_std = v_loa_102.trace().mean(), v_loa_102.trace().std()

z_hia_101_mean, z_hia_101_std = z_hia_101.trace().mean(), z_hia_101.trace().std()
z_hia_102_mean, z_hia_102_std = z_hia_102.trace().mean(), z_hia_102.trace().std()

z_loa_101_mean, z_loa_101_std = z_loa_101.trace().mean(), z_loa_101.trace().std()
z_loa_102_mean, z_loa_102_std = z_loa_102.trace().mean(), z_loa_102.trace().std()

In [5]:
# generate high anxiety data
n_hia = 54
n_loa = 61

n_mixed = 98
n_gain = 50



params_hia_101 = np.concatenate((np.random.normal(a_hia_101_mean, a_hia_101_std, size=(n_hia,1)), 
                                 np.random.normal(t_hia_101_mean, t_hia_101_std, size=(n_hia,1)),
                                 np.random.normal(v_hia_101_mean, v_hia_101_std, size=(n_hia,1)),
                                 np.random.normal(z_hia_101_mean, z_hia_101_std, size=(n_hia,1))), axis=1)

params_hia_102 = np.concatenate((np.random.normal(a_hia_102_mean, a_hia_102_std, size=(n_hia,1)), 
                                 np.random.normal(t_hia_102_mean, t_hia_102_std, size=(n_hia,1)),
                                 np.random.normal(v_hia_102_mean, v_hia_102_std, size=(n_hia,1)),
                                 np.random.normal(z_hia_102_mean, z_hia_102_std, size=(n_hia,1))), axis=1)

params_loa_101 = np.concatenate((np.random.normal(a_loa_101_mean, a_loa_101_std, size=(n_loa,1)), 
                                 np.random.normal(t_loa_101_mean, t_loa_101_std, size=(n_loa,1)),
                                 np.random.normal(v_loa_101_mean, v_loa_101_std, size=(n_loa,1)),
                                 np.random.normal(z_loa_101_mean, z_loa_101_std, size=(n_loa,1))), axis=1)

params_loa_102 = np.concatenate((np.random.normal(a_loa_102_mean, a_loa_102_std, size=(n_loa,1)), 
                                 np.random.normal(t_loa_102_mean, t_loa_102_std, size=(n_loa,1)),
                                 np.random.normal(v_loa_102_mean, v_loa_102_std, size=(n_loa,1)),
                                 np.random.normal(z_loa_102_mean, z_loa_102_std, size=(n_loa,1))), axis=1)

data_hia = pd.DataFrame()
data_loa = pd.DataFrame()

for i in range(n_hia):

    tmp_101, _ = hddm.generate.gen_rand_data({'101': {'a':params_hia_101[i,0], 
                                                      't':params_hia_101[i,1], 
                                                      'v':params_hia_101[i,2], 
                                                      'z':params_hia_101[i,3]}},
                                              size=n_mixed)

    tmp_102, _ = hddm.generate.gen_rand_data({'102': {'a':params_hia_102[i,0], 
                                                      't':params_hia_102[i,1], 
                                                      'v':params_hia_102[i,2], 
                                                      'z':params_hia_102[i,3]}},
                                              size=n_gain)

    data_subj = tmp_101.append(tmp_102, ignore_index=True).sample(frac=1).reset_index()

    data_subj['subj_idx'] = i
    
    data_hia = data_hia.append(data_subj, ignore_index=True)


for i in range(n_loa):

    tmp_101, _ = hddm.generate.gen_rand_data({'101': {'a':params_loa_101[i,0], 
                                                      't':params_loa_101[i,1], 
                                                      'v':params_loa_101[i,2], 
                                                      'z':params_loa_101[i,3]}},
                                              size=n_mixed)

    tmp_102, _ = hddm.generate.gen_rand_data({'102': {'a':params_loa_102[i,0], 
                                                      't':params_loa_102[i,1], 
                                                      'v':params_loa_102[i,2], 
                                                      'z':params_loa_102[i,3]}},
                                              size=n_gain)

    data_subj = tmp_101.append(tmp_102, ignore_index=True).sample(frac=1).reset_index(drop=True)
    data_subj['subj_idx'] = i
    data_loa = data_loa.append(data_subj, ignore_index=True)

In [12]:
np.savetxt('../parameter_recovery/params_hia_101.csv', params_hia_101, header="a,t,v,z", delimiter=',', fmt='%f')
np.savetxt('../parameter_recovery/params_hia_102.csv', params_hia_102, header="a,t,v,z", delimiter=',', fmt='%f')
np.savetxt('../parameter_recovery/params_loa_101.csv', params_loa_101, header="a,t,v,z", delimiter=',', fmt='%f')
np.savetxt('../parameter_recovery/params_loa_102.csv', params_loa_102, header="a,t,v,z", delimiter=',', fmt='%f')


data_hia = data_hia.rename(columns={'condition':'trial_type'})
data_loa = data_loa.rename(columns={'condition':'trial_type'})

data_hia.to_csv('../parameter_recovery/data_hia.csv')
data_loa.to_csv('../parameter_recovery/data_loa.csv')